# Классификатор ориентации документов

**Цель работы** — разработать решение для классификации ориентации документов на фото для последующей интеграции в готовый микросервис по их OCR-распознаванию. Классификатор должен определять одну из четырёх ориентаций для осуществления при необходимости соответствующего приведения изображения к классической ориентации. Данная операция необходима, так как модель сервиса для последующего распознавания справок работает только с изображениями с указанной ориентацией. Модель же, поддерживающая одновременно все из них, показала в среднем худшее качество распознавания текста по сравнению с исходной.

В наличии имеется по 344 копии различных медицинских справок в каждой из 4 ориентаций.

**Ход работы**

Для начала определимся с методом получения векторных представлений, эмбеддингов входящих изображений, подготовим их.

Далее выберем непосредственно классификатор, обучим его на имеющихся данных. После чего в итоге оценим качество его работы на тестовой выборке.
 
Таким образом работа будет состоять из следующих этапов:

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Получение-эмбеддингов-изображений" data-toc-modified-id="Получение-эмбеддингов-изображений-1">Получение эмбеддингов изображений</a></span></li><li><span><a href="#Обучение-классификатора" data-toc-modified-id="Обучение-классификатора-2">Обучение классификатора</a></span></li><li><span><a href="#Проверка-качества-на-тестовых-данных" data-toc-modified-id="Проверка-качества-на-тестовых-данных-3">Проверка качества на тестовых данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4">Вывод</a></span></li></ul></div>

## Получение эмбеддингов изображений

In [1]:
import os
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
import plotly.io as pio
from plotly.figure_factory import create_annotated_heatmap

from sklearn.metrics import \
    f1_score, \
    roc_auc_score, \
    accuracy_score, \
    precision_score, \
    recall_score, \
    confusion_matrix
from sklearn.model_selection import train_test_split

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from transformers import AutoModel

from skorch.classifier import NeuralNetClassifier
from skorch.callbacks import EpochScoring, EarlyStopping, LRScheduler

In [2]:
HOME = '/home/vladislav/ds/projects/pet/donorsearch_screenshots'
pio.renderers.default = 'notebook_connected'
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.use_deterministic_algorithms(True)
SEED = 345678
set_seed(SEED)

Для начала соберём датафрейм с путями к файлам и таргетом класса для получения эмбеддингов фотографий и их последующей классификации.

In [3]:
subfolders = ['top', 'right', 'left', 'bottom']
images = []
labels = []

label_mapping = {'top': 0, 'right': 1, 'left': 2, 'bottom': 3}

for subfolder in subfolders:
    path = os.path.join(HOME, subfolder)
    for file in os.listdir(path):
        images.append(os.path.join(subfolder, file))
        labels.append(label_mapping[subfolder])

orientations = pd.DataFrame({'images': images, 'label': labels}).sample(frac=1, random_state=SEED).reset_index(drop=True)
orientations.sample(10, random_state=SEED)

,images,label
6,right/e2df6e921ec5453e8d00b34ca26948bc.jpg,1
718,left/2b123493ea3447e8a78cd77864e8dad8.jpg,2
459,right/d0407f45eeab4d048babd75cf9b87570.jpg,1
129,top/7cd69c4019a64858ba58dce8ed3ce1a8.jpg,0
149,left/912bf69198e945e3afd97a90839df4c7.jpg,2
123,bottom/e22a4589af944ede90750c5ab0a44b07.jpg,3
365,left/515f2081d16046ec94e59c6bcb1290e1.jpg,2
177,bottom/8d3fee13bd3f4bc58aed10b12986c5dc.jpg,3
401,top/788599452a194797a78525003923d5d4.jpg,0
268,top/12bce2f3e2df49419b4f223574720dc2.jpg,0


Переопределим класс Dataset из PyTorch для подгрузки изображений и напишем функцию для генерации эмбеддингов. Используем чуть больший размер изображения при масштабировании с помощью **Resize()**, чем при его окончательной обрезке, для сохранения большего количества информации.

In [4]:
class DFDataset(Dataset):
    def __init__(self, df, dir, transform=None):
        self.df = df
        self.dir = dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        file = row.images
        path = os.path.join(self.dir, file)
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

def obtaining_img_embeddings(df, dir, model, batch_size):
    preprocess = transforms.Compose([
        transforms.Resize(384),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]
            )
        ])
    dataset = DFDataset(df, dir, transform=preprocess)
    loader = DataLoader(dataset, batch_size=batch_size)
    image_embeddings = []

    with torch.no_grad():
        for batch in loader:
            output = model(batch)
            image_embeddings.append(output[0][:,0,:].numpy())
            
    return np.concatenate(image_embeddings)

Эмбеддинги получим с помощью предобученной BEiT-модели (бертоподобного ViT) от Microsoft — https://huggingface.co/google/vit-base-patch16-224, во время экспериментов она лучше себя показала для данной задачи.

In [5]:
beit = AutoModel.from_pretrained('microsoft/beit-base-patch16-224-pt22k-ft22k')
with open(os.path.join(HOME, 'beit_encoder.pkl'), 'wb') as f:
    pickle.dump(beit, f)
with open(os.path.join(HOME, 'beit_encoder.pkl'), 'rb') as f:
    beit = pickle.load(f)

base_ids = orientations['images'].apply(
    lambda x: x.split('/')[-1].rsplit('.', 1)[0]
    )
train_ids, test_ids = train_test_split(
    base_ids.unique(),
    test_size=0.3,
    random_state=SEED
    )
train_df = orientations[base_ids.isin(train_ids)]
test_df = orientations[base_ids.isin(test_ids)]

train_features = obtaining_img_embeddings(train_df, HOME, beit, 128)
test_features = obtaining_img_embeddings(test_df, HOME, beit, 128)

train_target = train_df['label'].values
test_target = test_df['label'].values

print(
    'Размеры тренировочной и тестовой выборок с эмбеддингами: ',
    train_features.shape,
    test_features.shape,
    '\n',
    'Размеры тренировочной и тестовой выборок с целевым признаком: ',
    train_target.shape,
    test_target.shape,
    sep=''
    )

Размеры тренировочной и тестовой выборок с эмбеддингами: (960, 768)(416, 768)
Размеры тренировочной и тестовой выборок с целевым признаком: (960,)(416,)


Окончательно подготовим данные к обучению, представив их как тензоры PyTorch.

In [6]:
f_train = torch.FloatTensor(train_features)
f_test = torch.FloatTensor(test_features)

t_train = torch.LongTensor(train_target)
t_test = torch.LongTensor(test_target)

## Обучение классификатора

В качестве классификатора используем обычную FFNN c парой скрытых слоёв на базе классификатора из библиотеки Skorch. Значения различных гиперпараметров были подобраны ранее с помощью кросс-валидации.

Ориентироваться при обучении в первую очередь будем на макро F1-меру, так как нам важно то, насколько качественно будет определён каждый класс по отдельности, потому что в дальнейшем в рамках работы всего микросервиса мы будем переворачивать изображение фиксированным образом в зависимости от предсказания.

In [7]:
class Net(nn.Module):
    def __init__(
        self,
        n_hidden_neurons_1=861,
        n_hidden_neurons_2=141,
        n_in_neurons=768,
        n_out_neurons=4
        ):
        super().__init__()

        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.bn1 = nn.BatchNorm1d(n_hidden_neurons_1)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.bn2 = nn.BatchNorm1d(n_hidden_neurons_2)
        self.act2 = nn.ReLU()
        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

net = NeuralNetClassifier(
    module=Net,
    verbose=0,
    lr=5e-2,
    batch_size=-1,
    max_epochs=1000,
    optimizer=optim.NAdam,
    optimizer__eps=1e-06,
    callbacks=[
        EpochScoring(
            scoring='f1_macro',
            lower_is_better=False,
            name='F1'
            ),
        EarlyStopping(
            lower_is_better=False,
            monitor='F1',
            patience=55,
            load_best=True
            ),
        LRScheduler(
            policy=optim.lr_scheduler.CosineAnnealingWarmRestarts,
            monitor='F1',
            T_0=45
            )
        ]
    )

In [8]:
net.fit(f_train, t_train)
with open(os.path.join(HOME, 'skorch_orientation_classifier.pkl'), 'wb') as f:
    pickle.dump(net, f)
with open(os.path.join(HOME, 'skorch_orientation_classifier.pkl'), 'rb') as f:
    net = pickle.load(f)

## Проверка качества на тестовых данных

Посчитаем как истинность прогнозов нашей модели для всех классов по отдельности, так и различные классификационные метрики на тестовой выборке.

In [9]:
preds = net.predict(f_test)
conf_mx = pd.DataFrame(confusion_matrix(t_test, preds))

fig = create_annotated_heatmap(
    conf_mx.to_numpy().round(3),
    x=['top', 'right', 'left', 'bottom'],
    y=['top', 'right', 'left', 'bottom'],
    colorscale='Bluered',
    font_colors=['white']
    ).update_layout(
        font_size=15,
        width=1000,
        height=500,
        xaxis_title='Предсказания',
        yaxis_title='Истинные значения',
        title=dict(
            text='Марица ошибок на тестовых данных',
            x=.5,
            y=.95
            )
        ).update_xaxes(title_standoff=5)
fig.show()
print()
print(
    'Precision на тестовых данных',
    precision_score(t_test, preds, average='macro').round(3)
    )
print(
    'Recall на тестовых данных',
    recall_score(t_test, preds, average='macro').round(3)
    )
print(
    'F-1 на тестовых данных',
    f1_score(t_test, preds, average='macro').round(3)
    )
print(
    'Accuracy на тестовых данных',
    accuracy_score(t_test, preds).round(3)
    )
print(
    'AUC-ROC на тестовых данных',
    roc_auc_score(
        t_test,
        net.predict_proba(f_test),
        multi_class='ovr',
        average='macro'
        ).round(3)
    )


Precision на тестовых данных 0.927
Recall на тестовых данных 0.925
F-1 на тестовых данных 0.926
Accuracy на тестовых данных 0.925
AUC-ROC на тестовых данных 0.984


## Вывод

Разработанный классификатор достаточно хорошо справляется с определением ориентации медицинских документов, показатели всех метрик от 0.925. В итоге в контексте работы сервиса только в районе 8% изображений будут ошибочно не обработаны итоговой моделью распознавания.